In [1]:
import pandas as pd
import plotly.express as px
import geopandas as gpd
from dash import Dash, dcc, html, Output, Input
import dash_bootstrap_components as dbc

#CUSTOM GRAPHS
import ipynb.fs.full.GraphFunctions as MyGraphs

## Prep Pandas Dataframe

In [2]:
df_Raw = MyGraphs.GetDataframe()
#df_Raw.head()

## Prep GeoJson

In [3]:
gj_PhMap = MyGraphs.GetGeoJson()
#gj_PhMap.head()

# Website Development
### Stylesheet

In [4]:
external_CSS = [dbc.themes.MINTY, 'https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=[external_CSS])

# Components

#### [1] Income Choropleth

In [5]:
grph_1 = dcc.Graph(id='Choropleth_Income', figure={}, clickData=None, className='six colummns')
dpdn_1 = dcc.Dropdown(id='Choropleth_Dropdown', value='Mean', 
                      options=['Mean','Median', 'Min', 'Max'])

#### [2] Top 10 Jobs

In [6]:
grph_2 = dcc.Graph(id='Pie_TopJobs', figure={}, className='six colummns')

#### [3] Income vs Expenses

In [7]:
dpdn_3 = dcc.Dropdown(id='Dropdown_RegionIncomeVsExpenses', options=df_Raw['Region'].unique(), value=df_Raw['Region'].unique(), clearable=False, multi=True)
grph_3 = dcc.Graph(id='Bar_IncomeVsExpenses', figure={})

#### [4] Food Expenditure Breakdown

In [16]:
checklist_4 = dcc.Checklist(id='Food_Checklist', value= MyGraphs.GetFoodLabels(), 
                            options = MyGraphs.GetFoodLabels(),
                            inline=True)
grph_4 = dcc.Graph(id='Bar_FoodBreakdown', figure={})

# Layout

In [9]:
app.layout = html.Div([
    dpdn_1,
    html.Div([grph_1, grph_2]),

    dpdn_3, grph_3,
    
    checklist_4,
    grph_4
])

# Callback

#### [1] Income Choropleth

In [10]:
@app.callback(
    Output(component_id='Choropleth_Income', component_property='figure'),
    Input(component_id='Choropleth_Dropdown', component_property='value')
)
def UpdateIncomeChoropleth(dropdownChoice):
    fig = MyGraphs.GetFig_Choropleth(
        ref_Dataframe = df_Raw, 
        ref_GeoJson = gj_PhMap, 
        agg_Mode = dropdownChoice
    )
    return fig

#### [2] Top 10 Jobs

In [11]:
@app.callback(
    Output(component_id='Pie_TopJobs', component_property='figure'),
    Input(component_id='Choropleth_Income', component_property='clickData')
)
def UpdateTopJobs(clk_data):
        if clk_data is None:
            fig2 = MyGraphs.GetFig_MostEmployedJobs(df_Raw, 'NCR')
            return fig2
        else:
            clk_Region = clk_data['points'][0]['location']
            fig2 = MyGraphs.GetFig_MostEmployedJobs(df_Raw, clk_Region)
            return fig2

#### [3] Income vs Expenses

In [12]:
@app.callback(
    Output(component_id='Bar_IncomeVsExpenses', component_property='figure'),
    Input(component_id='Dropdown_RegionIncomeVsExpenses', component_property='value')
)
def UpdateIncomeExpenseComparison(slct_Regions):
    return MyGraphs.GetFig_IncomeVsExpenses(df_Raw, slct_Regions)

#### [4] Food Expenditure Breakdown

In [13]:
@app.callback(
    Output(component_id='Bar_FoodBreakdown', component_property='figure'),
    Input(component_id='Food_Checklist', component_property='value')
)
def UpdateFoodExpenditure(slct_foods):
    if slct_foods is None:
        return
    else:
        fig = MyGraphs.GetFig_FoodBreakdown(df_Raw, slct_foods)
        return fig

# Run

In [14]:
if __name__ == '__main__':
    app.run_server(port=8052)